# Bab 8 Backpropagation (2)

## Praktikum

### a) Fungsi *Training* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [137]:
import numpy as np

def sig(x):
    return 1 / (1 + np.exp(-x))

def sigd(x):
    s = sig(x)
    return s * (1 - s)

def bp_fit(X, target, layer_conf, max_epoch, max_error=.1, learn_rate=.1, print_per_epoch=100):
    np.random.seed(1)
    nin = [np.empty(i) for i in layer_conf]

    n = [np.empty(j + 1) if i < len(layer_conf) - 1 else np.empty(j) for i, j in enumerate(layer_conf)]
    w = [np.random.rand(layer_conf[i] + 1, layer_conf[i + 1]) for i in range(len(layer_conf) - 1)]
    dw = [np.empty((layer_conf[i] + 1, layer_conf[i + 1])) for i in range(len(layer_conf) - 1)]
    d = [np.empty(s) for s in layer_conf[1:]]
    din = [np.empty(s) for s in layer_conf[1:-1]]
    epoch = 0
    mse = 1

    for i in range(0, len(n) - 1):
        n[i][-1] = 1
        
    while (max_epoch == -1 or epoch < max_epoch) and mse > max_error:
        epoch += 1
        mse = 0
        for r in range(len(X)):
            n[0][:-1] = X[r]
            for L in range(1, len(layer_conf)):
                nin[L] = np.dot(n[L - 1], w[L - 1])
                n[L][:len(nin[L])] = sig(nin[L])
            e = target[r] - n[-1]
            mse += sum(e ** 2)
            d[-1] = e * sigd(nin[-1])
            dw[-1] = learn_rate * d[-1] * n[-2].reshape((-1, 1))

            for L in range(len(layer_conf) - 1, 1, -1):
                din[L - 2] = np.dot(d[L - 1], np.transpose(w[L - 1][:-1]))
                d[L - 2] = din[L - 2] * np.array(sigd(nin[L - 1]))

                dw[L - 2] = (learn_rate * d[L - 2]) * n[L - 2].reshape((-1, 1))

            for i in range(len(w)):
                w[i] += dw[i]
        mse /= len(X)
        if print_per_epoch > -1 and epoch % print_per_epoch == 0:
            print(f'Epoch {epoch}, MSE: {mse}')
    return w, epoch, mse

def onehot_enc(lbl, min_val=0):
    mi = min(lbl)
    enc = np.full((len(lbl), max(lbl) - mi + 1), min_val, np.int8)
    for i, x in enumerate(lbl):
        enc[i, x - mi] = 1
    return enc

def onehot_dec(enc, mi=0):
    return [np.argmax(e) + mi for e in enc]

### b) Fungsi *Testing* Backpropagation

Tulis kode ke dalam *cell* di bawah ini:

In [138]:
def bp_predict(X, w):
    n = [np.empty(len(i)) for i in w]
    nin = [np.empty(len(i[0])) for i in w]
    predict = []
    n.append(np.empty(len(w[-1][0])))

    for x in X:
        n[0][:-1] = x  
        for L in range(len(w)):
            nin[L] = np.dot(n[L], w[L])
            n[L + 1][:len(nin[L])] = sig(nin[L]) 

        predict.append(n[-1].copy())  

    return predict


### c) Percobaan Klasifikasi Dataset Iris

![Iris Dataset](https://www.spataru.at/images/blog/iris-dataset-svm/iris_types.jpg)

Tulis kode ke dalam *cell* di bawah ini:

In [144]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import accuracy_score


# Menggunakan dataset Iris
iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

# Membagi dataset menjadi data latih dan data uji dengan rasio 70:30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

# Mengatur parameter yang diminta
layer_conf = (4, 2, 3)  # 4 input neurons, 2 hidden neurons, 3 output neurons
max_epoch = 100
learn_rate = 0.1
max_error = 0.5

# Melatih model dengan parameter yang telah ditentukan
w, ep, mse = bp_fit(X_train, y_train, layer_conf=layer_conf, learn_rate=learn_rate, max_epoch=max_epoch, max_error=max_error, print_per_epoch=25)

# Menampilkan hasil pelatihan
print(f'Epochs: {ep}, MSE: {mse}')

# Melakukan prediksi pada data uji
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)

# Menghitung akurasi
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 25, MSE: 0.5587148548510855
Epochs: 29, MSE: 0.49569003845261594
Output: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.28888888888888886


In [148]:
# Mengatur parameter yang diminta
layer_conf = (4, 25, 3)  # 4 input neurons, 25 hidden neurons, 3 output neurons
max_epoch = 10000
learn_rate = 0.1
max_error = 0.01

iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

# Membagi dataset menjadi data latih dan data uji dengan rasio 70:30
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

# Melatih model dengan parameter yang telah ditentukan
w, ep, mse = bp_fit(X_train, y_train, layer_conf=layer_conf, learn_rate=learn_rate, max_epoch=max_epoch, max_error=max_error, print_per_epoch=1000)

# Menampilkan hasil pelatihan
print(f'Epochs: {ep}, MSE: {mse}')

# Melakukan prediksi pada data uji
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)

# Menghitung akurasi
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)


Epoch 1000, MSE: 0.042491178822878735
Epoch 2000, MSE: 0.04042211724471031
Epoch 3000, MSE: 0.03959534704315792
Epoch 4000, MSE: 0.03830921024857605
Epoch 5000, MSE: 0.03518870917492082
Epoch 6000, MSE: 0.028683001298715743
Epoch 7000, MSE: 0.023595119166769415
Epoch 8000, MSE: 0.02128192458419396
Epoch 9000, MSE: 0.020204288307448054
Epoch 10000, MSE: 0.019620449237882327
Epochs: 10000, MSE: 0.019620449237882327
Output: [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 1, 2, 1]
True : [0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0, 1, 2, 2, 0, 2, 2, 1]
Accuracy: 0.9555555555555556


In [ ]:
iris = datasets.load_iris()
X = minmax_scale(iris.data)
Y = onehot_enc(iris.target)

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=.3,random_state=1)

best_accuracy = 0
best_params = {}

layer_conf_options = [(4, 5, 3), (4, 15, 3), (4, 25, 3)]
max_epoch_options = [100, 1000, 10000]
max_error_options = [0.05, 0.01, 0.005]

for layer_conf in layer_conf_options:
    for max_epoch in max_epoch_options:
        for max_error in max_error_options:
            w, ep, mse = bp_fit(X_train, y_train, layer_conf=layer_conf, max_epoch=max_epoch, max_error=max_error)
            predict = bp_predict(X_test, w)
            predict = onehot_dec(predict)
            y_test = onehot_dec(y_test)
            accuracy = accuracy_score(predict, y_test)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['layer_conf'] = layer_conf
                best_params['max_epoch'] = max_epoch
                best_params['max_error'] = max_error
                best_params['learn_rate'] = 0.1
                best_params['accuracy'] = accuracy
            
                print(f'Layer Conf: {layer_conf}, Max Epoch: {max_epoch}, Max Error: {max_error}, Accuracy: {accuracy}')
                    
                    
print('Best Accuracy:', best_accuracy)
print('Best Params:', best_params)
w, ep, mse = bp_fit(X_train, y_train, layer_conf=best_params['layer_conf'],learn_rate=best_params['learn_rate'], max_epoch=best_params['max_epoch'], max_error=best_params['max_error'], print_per_epoch=25)
print(f'Epochs: {ep}, MSE: {mse}')
predict = bp_predict(X_test, w)
predict = onehot_dec(predict)
y_test = onehot_dec(y_test)
accuracy = accuracy_score(predict, y_test)
print('Output:', predict)
print('True :', y_test)
print('Accuracy:', accuracy)

Epoch 100, MSE: 0.18785482392338607
Layer Conf: (4, 5, 3), Max Epoch: 100, Max Error: 0.05, Accuracy: 0.6
Epoch 100, MSE: 0.18785482392338607
Epoch 100, MSE: 0.18785482392338607
Epoch 100, MSE: 0.18785482392338607
Epoch 200, MSE: 0.06884237603253064
Epoch 300, MSE: 0.0528378911150412
Epoch 100, MSE: 0.18785482392338607
Epoch 200, MSE: 0.06884237603253064
Epoch 300, MSE: 0.0528378911150412
Epoch 400, MSE: 0.04734469107899502
Epoch 500, MSE: 0.04456053723865837
Epoch 600, MSE: 0.04292953575851117
Epoch 700, MSE: 0.04190156923320104
Epoch 800, MSE: 0.041219770607664456
Epoch 900, MSE: 0.040747801007256224
Epoch 1000, MSE: 0.040408206226858645
Epoch 100, MSE: 0.18785482392338607
Epoch 200, MSE: 0.06884237603253064
Epoch 300, MSE: 0.0528378911150412
Epoch 400, MSE: 0.04734469107899502
Epoch 500, MSE: 0.04456053723865837
Epoch 600, MSE: 0.04292953575851117
Epoch 700, MSE: 0.04190156923320104
Epoch 800, MSE: 0.041219770607664456
Epoch 900, MSE: 0.040747801007256224
Epoch 1000, MSE: 0.04040820